In [1]:
import pandas as pd
import numpy as np

# Calcular Principais Indicadores - LV

- Carregar dados estimativa populacional e de casos confirmados



## Casos Confirmados

O Power BI consegue criar mapas tendo os nomes (município, estado, país), então não será necessário o código IBGE correto, 7 dígitos, ou dados de geolocalização.

In [2]:
confirmados = pd.read_csv('../data/interim/leivis/casos_confirmados_full_limpos.csv', low_memory=False)

## População Estimada por Município/Ano
- A partir de 2017 os códigos passaram a ter 7 dígitos. Um município com 6 passou a ter 7 e está sendo considerado um novo. Corrigir. Basta retirar o último dígito, verificador.

- Alguns municípios, 6 deles, não tem estimativa populacional entre 2007 e 2011. Provavelmente por não terem tido senso nestes anos, recém criados. 

In [3]:
populacao = pd.read_csv('../data/interim/populacao/output_full.csv')
populacao.MUNIC_RES = populacao.MUNIC_RES.apply(lambda x: int(str(x)[:6]))

In [4]:
populacao = pd.pivot_table(populacao, values='POPULACAO', index='MUNIC_RES', columns='ANO')

In [5]:
# verificando municípios que possuem algum ano sem estimativa, totalizando 6
populacao.loc[populacao.isnull().sum(axis=1) > 0,:]

ANO,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
MUNIC_RES,,,,,,,,,,,,,
150475,NaN,NaN,NaN,NaN,NaN,15018.0,15232.0,15341.0,15446.0,15548.0,15646.0,15982.0,16084.0
220672,NaN,8501.0,7895.0,8039.0,8149.0,8227.0,8289.0,8330.0,8366.0,8398.0,8425.0,8536.0,8570.0
421265,NaN,NaN,NaN,NaN,NaN,9416.0,9687.0,9761.0,9835.0,9908.0,9980.0,10022.0,10091.0
422000,NaN,NaN,NaN,NaN,NaN,11136.0,11628.0,11824.0,12018.0,12212.0,12403.0,12570.0,12760.0
431454,NaN,NaN,NaN,NaN,NaN,2578.0,2681.0,2800.0,2824.0,2847.0,2868.0,2968.0,3003.0
500627,NaN,NaN,NaN,NaN,NaN,4723.0,4942.0,5047.0,5150.0,5251.0,5350.0,5455.0,5555.0


In [6]:
populacao.reset_index().to_csv('../data/processed/populacao.csv', index=False)

## Informações sobre Municípios

In [7]:
municipios = pd.read_excel('../data/raw/IBGE_BRASIL_MUNICIPIO.xls')
municipios = municipios.loc[:, ['Código Município Completo','Nome_Município', 'Nome_UF', 'Nome_Mesorregião', 'Nome_Microrregião']]
municipios.columns = ['ibge_code', 'municipio','estado','mesorregiao','microrregiao']

municipios.ibge_code = municipios.ibge_code.apply(lambda x: str(x)[:6])
municipios

,ibge_code,municipio,estado,mesorregiao,microrregiao
0,110001,Alta Floresta D'Oeste,Rondônia,Leste Rondoniense,Cacoal
1,110037,Alto Alegre dos Parecis,Rondônia,Leste Rondoniense,Cacoal
2,110040,Alto Paraíso,Rondônia,Leste Rondoniense,Ariquemes
3,110034,Alvorada D'Oeste,Rondônia,Leste Rondoniense,Alvorada D'Oeste
4,110002,Ariquemes,Rondônia,Leste Rondoniense,Ariquemes
...,...,...,...,...,...
5565,522200,Vianópolis,Goiás,Sul Goiano,Pires do Rio
5566,522205,Vicentinópolis,Goiás,Sul Goiano,Meia Ponte
5567,522220,Vila Boa,Goiás,Leste Goiano,Entorno de Brasília
5568,522230,Vila Propício,Goiás,Leste Goiano,Entorno de Brasília


In [8]:
municipios.to_csv('../data/processed/municipios.csv', index=False, encoding='utf-8')
municipios.ibge_code = municipios.ibge_code.astype('string')

## Indicadores

- O cálculo dos indicadores é feito de acordo com as instruções do `CADERNO DE INDICADORES DE LEISHMANIOSE TEGUMENTAR E LEISHMANIOSE VISCERAL`, 2018, MINISTÉRIO DA SAÚDE, Secretaria de Vigilância em Saúde, presente no diretório de Referências.

In [9]:
import sys
sys.path.append('../')

import src.indicadores as indicadores

In [10]:
output = '../data/indicadores/visceral/{}'

In [11]:
help(indicadores.total_casos_novos_municipio_infeccao_ano_14)

Help on function total_casos_novos_municipio_infeccao_ano_14 in module src.indicadores:

total_casos_novos_municipio_infeccao_ano_14(df)
    | Importância do indicador: Conhecer a ocorrência de casos de LV, a distribuição espacial e temporal e
    | a tendência;
    |
    |Método de cálculo: Número total de casos novos de LV por local provável de infecção (UF, município, região administrativa ou localidade) no ano de notificação
    | input:
    | - df (dataframe com dados históricos)
    | - local (UF, município, região administrativa ou localidade)
    | - ano (2007 a 2019)
    | output:
    | - dataframe com Número total de casos novos de LV por local provável de infecção.



### A partir daqui serão calculados 12 indicadores de LV

In [12]:
indicadores.total_casos_novos_municipio_infeccao_ano_14(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_INF'}
).fillna(0).to_csv(output.format('indicador_14_total_casos_novos_municipio_infeccao.csv'), index=False)

In [13]:
indicadores.taxa_geral_incidencia_municipio_infeccao_ano_15(
    confirmados, populacao
).reset_index().rename(
    columns={'index': 'CO_MN_INF'}
).fillna(0).to_csv(output.format('indicador_15_taxa_incidencia_municipio_infeccao.csv'), index=False)

In [14]:
indicadores.proporcao_casos_confirmados_criterio_laboratorial_16(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_RESI'}
).fillna(0).to_csv(
    output.format('indicador_16_proporcao_casos_criterio_laboratorial_municipio_residencia.csv'), 
    index=False
)

In [15]:
indicadores.proporcao_casos_menor_5_anos_17(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_INF'}
).fillna(0).to_csv(
    output.format('indicador_17_proporcao_casos_menor_5_anos_municipio_infeccao.csv'), 
    index=False
)

In [16]:
indicadores.proporcao_casos_50_anos_mais_18(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_INF'}
).fillna(0).to_csv(
    output.format('indicador_18_proporcao_casos_50_anos_mais_municipio_infeccao.csv'), 
    index=False
)

In [17]:
indicadores.proporcao_casos_hiv_19(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_INF'}
).fillna(0).to_csv(
    output.format('indicador_19_proporcao_casos_hiv_municipio_infeccao.csv'), 
    index=False
)

In [18]:
indicadores.proporcao_casos_cura_clinica_20(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_RESI'}
).fillna(0).to_csv(
    output.format('indicador_20_proporcao_casos_cura_clinica_municipio_residencia.csv'), 
    index=False
)

In [19]:
indicadores.total_obitos_residencia_ano_21(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_RESI'}
).fillna(0).to_csv(
    output.format('indicador_21_total_obitos_municipio_residencia.csv'), 
    index=False
)

In [20]:
indicadores.taxa_letalidade_municipio_residencia_ano_22(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_RESI'}
).fillna(0).to_csv(
    output.format('indicador_22_taxa_letalidade_municipio_residencia.csv'), 
    index=False
)

In [21]:
indicadores.taxa_letalidade_hiv_municipio_residencia_ano_23(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_RESI'}
).fillna(0).to_csv(
    output.format('indicador_23_taxa_letalidade_hiv_municipio_residencia.csv'), 
    index=False
)

In [22]:
indicadores.taxa_letalidade_menor_5_anos_municipio_residencia_ano_24(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_RESI'}
).fillna(0).to_csv(
    output.format('indicador_24_taxa_letalidade_menor_5_anos_municipio_residencia.csv'), 
    index=False
)

In [23]:
indicadores.taxa_letalidade_50_anos_mais_municipio_residencia_ano_25(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_RESI'}
).fillna(0).to_csv(
    output.format('indicador_25_taxa_letalidade_50_anos_mais_municipio_residencia.csv'), 
    index=False
)

In [24]:
indicadores.proporcao_casos_evolucao_ignorada_branco_ano_26(
    confirmados
).reset_index().rename(
    columns={'index': 'CO_MN_RESI'}
).fillna(0).to_csv(
    output.format('indicador_26_proporcao_casos_evolucao_ignorada_branco_municipio_residencia.csv'), 
    index=False
)